TODO
* Merge Tables 2,4,10 to work together


Someday:
* Fix Italics:
    * Work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
    * Not work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
        
Learning: 
* Look for specific class types in BeautifulSoup. The fact that the transcript was formated as a *wikitable* made life much easier

In [1]:
# imports

#standard
import pandas as pd
#import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Data Pulls
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#Get Episode titles for Season 1

In [3]:
url_base = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
response = requests.get(url_base)

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

tables = soup.find_all("table")

print(f"Found {len(tables)} tables on the page.")

Found 90 tables on the page.


In [5]:
# Table 2: Book 1 Water
# Table 4: Book 2 Earth
# Table 10: Book 3 Fire

# if len(tables) > 0:
#     print("First table:")
#     print(tables[10].prettify())

In [6]:
episode_list = []

specific_table = soup.find_all("table")[2]  # Replace 0 with the index of the table you want
links = specific_table.find_all("a")
for link in links:
    episode_list.append(link.get("href"))

In [7]:
episode_list_filtered = [episode for episode in episode_list if "commentary" not in episode]

In [8]:
episode_list_filtered

['/wiki/Transcript:The_Boy_in_the_Iceberg',
 '/wiki/Transcript:The_Avatar_Returns',
 '/wiki/Transcript:The_Southern_Air_Temple',
 '/wiki/Transcript:The_Warriors_of_Kyoshi',
 '/wiki/Transcript:The_King_of_Omashu',
 '/wiki/Transcript:Imprisoned',
 '/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World',
 '/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku',
 '/wiki/Transcript:The_Waterbending_Scroll',
 '/wiki/Transcript:Jet_(episode)',
 '/wiki/Transcript:The_Great_Divide',
 '/wiki/Transcript:The_Storm',
 '/wiki/Transcript:The_Blue_Spirit',
 '/wiki/Transcript:The_Fortuneteller',
 '/wiki/Transcript:Bato_of_the_Water_Tribe',
 '/wiki/Transcript:The_Deserter',
 '/wiki/Transcript:The_Northern_Air_Temple',
 '/wiki/Transcript:The_Waterbending_Master',
 '/wiki/Transcript:The_Siege_of_the_North,_Part_1',
 '/wiki/Transcript:The_Siege_of_the_North,_Part_2']

In [9]:
### Get Transcript

In [10]:
base = 'https://avatar.fandom.com'

In [11]:
episode_list_filtered[0]

'/wiki/Transcript:The_Boy_in_the_Iceberg'

In [12]:
url_episode_list = []
for episode in episode_list_filtered:
    #print(base + episode)
    url_episode_list.append(base + episode)

for episode in url_episode_list:
    response = requests.get(episode_base)
    ep_soup = BeautifulSoup(response.content, "html.parser")
    tables = ep_soup.find_all("table")

In [13]:
#First Episode
episode_base = url_episode_list[0]
response = requests.get(episode_base)
ep_soup = BeautifulSoup(response.content, "html.parser")
#tables = ep_soup.find_all("table")

In [14]:
tables = ep_soup.select("table.wikitable")

data = []

# Step 4: Check if there are at least 2 tables and select the second one
if len(tables) > 1:
    second_table = tables[1]  # Index 1 corresponds to the second table
    # Now, find all rows within this second table's tbody
    rows = second_table.select("tbody tr")
    
    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')    # Find the <td> for dialogue
    
        if character_cell and dialogue_cell:  # Ensure both are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
            for italics in dialogue_cell.find_all('i'):
                # Get the text inside <i>, append '123' and replace the <i> with this text
                italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text
            
            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            
            data.append({'character': character, 'dialogue': dialogue})

            df = pd.DataFrame(data)
            
            #Cleaning
            df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()
            
            #Word Count
            df['word_count'] = df['dialogue'].str.split().apply(len)

In [15]:
pd.set_option('display.max_colwidth', None) 
df.head(20)

,character,dialogue,word_count
0,Sokka,"It's not getting away from me this time. Watch and learn, Katara. This is how you catch a fish.",19
1,Katara,"Sokka, look!",2
2,Sokka,"Sshh! Katara, you're going to scare it away. Mmmm ... I can already smell it cookin'.",16
3,Katara,"But, Sokka! I caught one!",5
4,Katara,Hey!,1
5,Sokka,"Why is it that every time you play with magic water, I get soaked?",14
6,Katara,"It's not ""magic"", it's waterbending! And it's ...",8
7,Sokka,"Yeah, yeah, ""an ancient art unique to our culture"", blah, blah, blah. Look, I'm just saying that if I had weird powers, I'd keep my weirdness to myself.",28
8,Katara,You're calling me weird? I'm not the one who makes muscles at myself every time I see my reflection in the water!,22
9,Katara,Watch out! Go left! Go left!,6


In [16]:
pd.set_option('display.max_colwidth', None) 
df.tail(20)

,character,dialogue,word_count
133,Aang,Whoa! What is that?,4
134,Katara,"A Fire Navy ship. And a very bad memory for my people. Aang, stop! We're not allowed to go near it! The ship could be booby-trapped!",26
135,Aang,"If you want to be a bender, you have to let go of fear.",14
136,Katara,This ship has haunted my tribe since Gran-Gran was a little girl. It was part of the Fire Nation's first attacks.,21
137,Aang,"Okay, back up. I have friends all over the world, even in the Fire Nation. I've never seen any war.",20
138,Katara,"Aang, how long were you in that iceberg?",8
139,Aang,"I don't know. A few days, maybe?",7
140,Katara,I think it was more like a hundred years!,9
141,Aang,What?! That's impossible! Do I look like a hundred-twelve-year-old man to you?,12
142,Katara,"Think about it. The War is a century old. You don't know about it because, somehow, you were in there the whole time! It's the only explanation.",27


In [17]:
# Table 1 - Intro
if len(tables) > 0:
    print("Second table:")
    print()
    print(tables[1].prettify())

Second table:

<table class="wikitable" width="100%">
 <tbody>
  <tr>
   <td>
   </td>
   <td>
    <i>
     As the title card fades, the scene opens onto a shot of an icy sea before panning slowly to the left, revealing more towering icebergs drifting in the water as the shot rotates, moving over a large snow-covered area where the untouched blanket of snow is broken by two tracks of footsteps. The shot fades to another shot of the sea and of icebergs in blue water contrasted by the paleness of the sky beyond. As the shot once again pans to the left and rotates likewise, a small canoe comes into view, its motion through the water indicated by the wake left behind it. The shot zooms slightly on the canoe before cutting down to it, providing a side angle of the canoe and of the walls of ice rising on either side of the vessel. Sokka, a teenage boy, and his younger sister, Katara, a teenage girl, are sitting in the boat; Sokka holds a spear at the ready, while Katara simply stares into th